# 🤖 Suivi du Projet Machine Learning - Segmentation Clients
## Prédiction de Réponse aux Campagnes Marketing

**Objectif :** Construire un modèle prédictif pour prédire la probabilité qu'un client réponde à une campagne future.

**Dataset :** ML_DataSet.csv (2237 clients × 49 features)

**Cible :** Reponse_Derniere_Campagne (0/1)

---

## 📊 Phase 1 : Exploration et Préparation des Données

In [ ]:
# Imports nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✅ Bibliothèques importées avec succès")
print(f"📅 Date de dernière mise à jour : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Chargement du dataset
df = pd.read_csv('ML_DataSet.csv')

print("=" * 70)
print("INFORMATIONS GÉNÉRALES DU DATASET")
print("=" * 70)
print(f"Nombre de lignes (clients) : {df.shape[0]}")
print(f"Nombre de colonnes (features) : {df.shape[1]}")
print(f"Taille mémoire : {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\n" + "=" * 70)

# Aperçu des données
df.head()

### 🎯 Analyse de la Variable Cible

In [ ]:
# Distribution de la variable cible
target_col = 'Reponse_Derniere_Campagne'
target_dist = df[target_col].value_counts()
target_pct = df[target_col].value_counts(normalize=True) * 100

print("=" * 70)
print("DISTRIBUTION DE LA VARIABLE CIBLE")
print("=" * 70)
print(f"Variable : {target_col}\n")
for classe in sorted(df[target_col].unique()):
    count = target_dist[classe]
    pct = target_pct[classe]
    print(f"  Classe {classe} : {count:4d} clients ({pct:.2f}%)")

ratio_desequilibre = target_dist[0] / target_dist[1]
print(f"\n⚠️  Ratio de déséquilibre : {ratio_desequilibre:.2f}:1")
if ratio_desequilibre > 3:
    print("   → Déséquilibre IMPORTANT détecté")
    print("   → Stratégie recommandée : scale_pos_weight, class_weight, ou SMOTE")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Barplot
axes[0].bar(['Non (0)', 'Oui (1)'], target_dist.values, color=['#e74c3c', '#2ecc71'])
axes[0].set_title('Distribution de la Variable Cible', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Nombre de clients')
axes[0].set_xlabel('Réponse à la campagne')
for i, v in enumerate(target_dist.values):
    axes[0].text(i, v + 30, str(v), ha='center', fontweight='bold')

# Pie chart
colors = ['#e74c3c', '#2ecc71']
axes[1].pie(target_dist.values, labels=['Non (0)', 'Oui (1)'], autopct='%1.1f%%', 
            startangle=90, colors=colors, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('Proportion des Réponses', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✅ Analyse de la variable cible terminée")

### 📋 Sélection des Features pour le Modèle

In [ ]:
# Colonnes à exclure du modèle
colonnes_a_exclure = [
    'ID_Client',                      # Identifiant
    'Annee_Naissance',                # Redondant avec Age_Inscription
    'Date_Inscription',               # Format date, redondant avec Jour_Inscription_Encode
    'Niveau_Education',               # Redondant avec Niveau_Education_Encode
    'Statut_Marital',                 # Redondant avec Statut_Marital_Encode
    'Statut_Marital_Texte',           # Redondant
    'Jour_Inscription',               # Redondant avec Jour_Inscription_Encode
    'Categorie_Age',                  # Redondant avec Categorie_Age_Encode
    'Cout_Contact_Z',                 # Constante (toujours 3)
    'Revenus_Z',                      # Constante (toujours 11)
    'Reponse_Derniere_Campagne',     # TARGET (à extraire séparément)
    'Enfants_Maison',                 # Redondant avec Total_Enfants
    'Ados_Maison'                     # Redondant avec Total_Enfants
]

print("=" * 70)
print("PRÉPARATION DES FEATURES")
print("=" * 70)
print(f"\nColonnes totales : {len(df.columns)}")
print(f"Colonnes à exclure : {len(colonnes_a_exclure)}")
print(f"Colonnes pour le modèle : {len(df.columns) - len(colonnes_a_exclure)}")

# Créer X et y
X = df.drop(columns=colonnes_a_exclure)
y = df['Reponse_Derniere_Campagne']

print(f"\n✅ X shape : {X.shape}")
print(f"✅ y shape : {y.shape}")

print("\n--- FEATURES SÉLECTIONNÉES ---")
for i, col in enumerate(X.columns, 1):
    print(f"{i:2d}. {col}")

# Vérifier les valeurs manquantes
missing = X.isnull().sum()
if missing.sum() > 0:
    print("\n⚠️  Valeurs manquantes détectées :")
    print(missing[missing > 0])
else:
    print("\n✅ Aucune valeur manquante détectée")

### 📊 Statistiques Descriptives des Features

In [ ]:
# Statistiques descriptives
print("=" * 70)
print("STATISTIQUES DESCRIPTIVES")
print("=" * 70)

X.describe().T

### 🔍 Analyse de Corrélation avec la Cible

In [ ]:
# Corrélation avec la variable cible
df_with_target = X.copy()
df_with_target['Target'] = y

correlations = df_with_target.corr()['Target'].drop('Target').sort_values(ascending=False)

print("=" * 70)
print("TOP 15 FEATURES CORRÉLÉES AVEC LA CIBLE")
print("=" * 70)
print(correlations.head(15))

print("\n" + "=" * 70)
print("TOP 15 FEATURES NÉGATIVEMENT CORRÉLÉES")
print("=" * 70)
print(correlations.tail(15))

# Visualisation
fig, ax = plt.subplots(figsize=(10, 8))
top_features = pd.concat([correlations.head(10), correlations.tail(10)])
colors = ['green' if x > 0 else 'red' for x in top_features.values]
top_features.plot(kind='barh', ax=ax, color=colors)
ax.set_title('Top 20 Features par Corrélation avec la Cible', fontsize=14, fontweight='bold')
ax.set_xlabel('Corrélation de Pearson')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.tight_layout()
plt.show()

print("\n✅ Analyse de corrélation terminée")

---
## 🤖 Phase 2 : Construction et Évaluation des Modèles

### Modèles à tester :
1. **Régression Logistique** (baseline)
2. **Random Forest** (robuste)
3. **XGBoost** (champion attendu)

In [ ]:
# Imports pour le machine learning
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    roc_auc_score, 
    roc_curve,
    precision_recall_curve,
    f1_score,
    accuracy_score
)

try:
    from xgboost import XGBClassifier
    xgboost_available = True
except ImportError:
    print("⚠️  XGBoost non installé. Installer avec : pip install xgboost")
    xgboost_available = False

print("✅ Bibliothèques ML importées avec succès")

### 🔀 Split Train/Test avec Stratification

In [ ]:
# Split stratifié (important pour le déséquilibre)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Maintient les proportions
)

print("=" * 70)
print("SPLIT TRAIN/TEST")
print("=" * 70)
print(f"Train set : {X_train.shape[0]} clients ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set  : {X_test.shape[0]} clients ({X_test.shape[0]/len(X)*100:.1f}%)")

print("\nDistribution dans Train set :")
print(y_train.value_counts())
print(f"Ratio : {y_train.value_counts()[0] / y_train.value_counts()[1]:.2f}:1")

print("\nDistribution dans Test set :")
print(y_test.value_counts())
print(f"Ratio : {y_test.value_counts()[0] / y_test.value_counts()[1]:.2f}:1")

# Normalisation (utile pour Régression Logistique)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ Données préparées pour l'entraînement")

### 📊 Fonction d'Évaluation des Modèles

In [ ]:
def evaluer_modele(model, X_train, X_test, y_train, y_test, nom_modele):
    """
    Évalue un modèle et affiche les métriques complètes
    """
    print("\n" + "=" * 70)
    print(f"ÉVALUATION : {nom_modele}")
    print("=" * 70)
    
    # Entraînement
    model.fit(X_train, y_train)
    
    # Prédictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Métriques
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    print(f"\n📊 Métriques Globales :")
    print(f"  Accuracy  : {accuracy:.4f}")
    print(f"  F1-Score  : {f1:.4f}")
    print(f"  ROC-AUC   : {roc_auc:.4f}")
    
    print(f"\n📋 Rapport de Classification :")
    print(classification_report(y_test, y_pred, target_names=['Non (0)', 'Oui (1)']))
    
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Matrice de confusion
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
                xticklabels=['Non (0)', 'Oui (1)'],
                yticklabels=['Non (0)', 'Oui (1)'])
    axes[0].set_title(f'Matrice de Confusion - {nom_modele}', fontweight='bold')
    axes[0].set_ylabel('Vraie Classe')
    axes[0].set_xlabel('Classe Prédite')
    
    # Courbe ROC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    axes[1].plot(fpr, tpr, label=f'{nom_modele} (AUC = {roc_auc:.3f})', linewidth=2)
    axes[1].plot([0, 1], [0, 1], 'k--', label='Aléatoire (AUC = 0.5)')
    axes[1].set_xlabel('Taux de Faux Positifs')
    axes[1].set_ylabel('Taux de Vrais Positifs')
    axes[1].set_title(f'Courbe ROC - {nom_modele}', fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'model': model,
        'accuracy': accuracy,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

print("✅ Fonction d'évaluation créée")

---
## 🎯 À COMPLÉTER : Entraînement des Modèles

### Modèle 1 : Régression Logistique (Baseline)

In [ ]:
# TODO: Entraîner et évaluer la Régression Logistique
# modele_lr = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
# resultats_lr = evaluer_modele(modele_lr, X_train_scaled, X_test_scaled, y_train, y_test, "Régression Logistique")

### Modèle 2 : Random Forest

In [ ]:
# TODO: Entraîner et évaluer Random Forest
# modele_rf = RandomForestClassifier(n_estimators=100, max_depth=10, class_weight='balanced', random_state=42)
# resultats_rf = evaluer_modele(modele_rf, X_train, X_test, y_train, y_test, "Random Forest")

### Modèle 3 : XGBoost (Champion)

In [ ]:
# TODO: Entraîner et évaluer XGBoost
# scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]
# modele_xgb = XGBClassifier(scale_pos_weight=scale_pos_weight, max_depth=5, learning_rate=0.1, n_estimators=100, random_state=42)
# resultats_xgb = evaluer_modele(modele_xgb, X_train, X_test, y_train, y_test, "XGBoost")

---
## 📊 Comparaison des Modèles

In [ ]:
# TODO: Comparer les performances des 3 modèles
# Créer un DataFrame de comparaison avec accuracy, f1_score, roc_auc

---
## 🔍 Interprétabilité : Feature Importance

In [ ]:
# TODO: Afficher l'importance des features du meilleur modèle
# Pour Random Forest et XGBoost : model.feature_importances_

---
## 📝 Conclusions et Prochaines Étapes

### Résultats obtenus :
- [ ] Modèle baseline (Régression Logistique) entraîné
- [ ] Random Forest entraîné
- [ ] XGBoost entraîné
- [ ] Comparaison des performances effectuée
- [ ] Feature importance analysée

### Améliorations possibles :
1. **Optimisation des hyperparamètres** (GridSearchCV, RandomizedSearchCV)
2. **SMOTE** pour rééquilibrer le dataset
3. **Feature Engineering** supplémentaire
4. **SHAP values** pour l'interprétabilité XGBoost
5. **Calibration** des probabilités
6. **Ensemble methods** (stacking, voting)

### Impact Business :
- Meilleur ciblage des campagnes marketing
- Optimisation du budget marketing
- Augmentation du ROI des campagnes

---

**Date de dernière mise à jour :** [À compléter]

**Auteur :** [Votre nom]

**Projet :** Rush 4 - Segmentation Clients